**Title: Automated Tile Extraction from WSIs within ROI Annotated with QuPath Using Python**

**Description:**

This Python script is a powerful tool designed to extract tiles from Whole Slide Images (WSIs) specifically within Regions of Interest (ROIs) annotated using the popular digital pathology software QuPath. It leverages the capabilities of the OpenSlide library to work with WSIs and the Shapely library to process geometrical data.

**Key Features:**

1. **WSI Compatibility:** This tool is compatible with Whole Slide Images (WSIs) in various formats, such as SVS.

2. **GeoJSON Support:** It reads and processes ROIs stored in GeoJSON format. The provided script assumes that the GeoJSON file contains polygon coordinates that define the ROI. These coordinates are extracted to create an ROI polygon.

3. **ROI-Based Extraction:** The script only extracts tiles that fall within the defined ROI polygon, allowing you to focus on specific areas of interest in the WSI.

4. **Customizable Parameters:** You can adjust parameters like the tile size and tissue threshold to suit your specific requirements.

5. **Output Management:** Extracted tiles are saved in an output directory with user-defined names for further analysis or visualization.

**Use Cases:**

- **Digital Pathology Research:** Ideal for researchers and pathologists working with digital pathology data to extract and analyze specific tissue regions.

- **QuPath Integration:** Seamlessly integrates with QuPath's ROI annotation feature, enabling you to work with your existing annotations.

- **Automation:** Offers an automated solution for tile extraction, streamlining the process and reducing manual effort.

**How to Use:**

1. Ensure you have the required Python libraries installed, such as OpenSlide and Shapely.

2. Prepare your QuPath annotations in GeoJSON format.

3. Customize parameters if needed, such as tile size and tissue threshold.

4. Run the script with the path to your WSI and GeoJSON file.

5. Extracted tiles will be saved in the specified output directory.

---

This code is a valuable asset for digital pathology professionals and researchers looking to extract and work with specific regions of interest within Whole Slide Images.

In [ ]:
import json
from openslide import OpenSlide
import numpy as np
import os
from shapely.geometry import Polygon

In [ ]:
# Load the GeoJSON file
with open('path/to/your_geojson_file.geojson') as f:
    geojson_data = json.load(f)

# Extract the coordinates of the ROI
coordinates = geojson_data['geometry']['coordinates'][0]

# Create a polygon from the coordinates using the shapely library
roi_polygon = Polygon(coordinates)

In [ ]:
wsi_path = "path/to/your_wsi_image.svs"
slide = OpenSlide(wsi_path)

tile_size = (256, 256)
level = 0
output_dir = "path/to/output_tiles"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
tiles = []

for y in range(0, slide.level_dimensions[level][1], tile_size[1]):
    for x in range(0, slide.level_dimensions[level][0], tile_size[0]):
        region = slide.read_region((x, y), level, tile_size)

        # Check if the tile is within the ROI polygon
        tile_bbox = Polygon(
            [(x, y), (x + tile_size[0], y), (x + tile_size[0], y + tile_size[1]), (x, y + tile_size[1])])

        if tile_bbox.intersects(roi_polygon):
            # Perform tissue detection here to determine if the tile has enough tissue
            tiles.append(region)

# Save the non-empty tiles
for i, tile in enumerate(tiles):
    tile.save(f"{output_dir}/tile_{i}.png")

slide.close()

# **License**

This Notebook has been released under the [Apache 2.0](http://www.apache.org/licenses/LICENSE-2.0) open source license.